
# Deep Learning Model Implementation for Binary Classification

This notebook implements a deep learning model for binary classification using a small numerical dataset. The model architecture will consist of dense layers with dropout and regularization to prevent overfitting.


In [50]:
# %conda install tensorflow
# Necessary imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l1_l2


In [51]:

# Load and preprocess the data
data = pd.read_csv('/home/matt/Projects/water-ml/datasets/labelled.csv')  # Update with actual path
data.replace('ND', 0, inplace=True)

# Encoding categorical features
categorical_columns = ['Taxa A1', 'Taxa A2', 'Taxa A3', 'Taxa A4', 'Taxa A5', 'Taxa B1', 'Taxa B2', 'Taxa B3']  # Update as needed
for col in categorical_columns:
    data[col] = data[col].astype(str)
    
encoder = OneHotEncoder(sparse=False)    
data_encoded = pd.DataFrame(encoder.fit_transform(data[categorical_columns]))
data_encoded.columns = encoder.get_feature_names_out(categorical_columns)
data.drop(categorical_columns ,axis=1, inplace=True)
data = pd.concat([data, data_encoded], axis=1)

# Separating features and target variable
X = data.drop(['Scheme', 'Sample'], axis=1)  # Update target and identifier columns as needed
y = data['Scheme'].map({'Stable': 0, 'Failure': 1})

# Normalizing the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


/home/matt/miniconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [52]:
from imblearn.over_sampling import SMOTE

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Augmenting the dataset with SMOTE
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [53]:

# Model architecture v1 (overfitting)
# model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(X_scaled.shape[1],), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

# Adjusted Model architecture
# model = Sequential()
# model.add(Dense(32, activation='relu', input_shape=(X_smote.shape[1],), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# # Reduced dropout rate
# model.add(Dropout(0.3))
# model.add(Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# # Removed one layer to simplify the model
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [54]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Adjusting the model architecture and parameters
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(X_smote.shape[1],), kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with a possibly adjusted learning rate
model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

# Implementing Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model training with Early Stopping
history = model.fit(X_smote, y_smote, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)

Epoch 1/100
7/7 [==============================] - 1s 46ms/step - loss: 1.3195 - accuracy: 0.4906 - val_loss: 1.3239 - val_accuracy: 0.4634
Epoch 2/100
7/7 [==============================] - 0s 22ms/step - loss: 1.2660 - accuracy: 0.4858 - val_loss: 1.2926 - val_accuracy: 0.5122
Epoch 3/100
7/7 [==============================] - 0s 15ms/step - loss: 1.1757 - accuracy: 0.6321 - val_loss: 1.2653 - val_accuracy: 0.5122
Epoch 4/100
7/7 [==============================] - 0s 13ms/step - loss: 1.1534 - accuracy: 0.6038 - val_loss: 1.2399 - val_accuracy: 0.5366
Epoch 5/100
7/7 [==============================] - 0s 16ms/step - loss: 1.1067 - accuracy: 0.6934 - val_loss: 1.2146 - val_accuracy: 0.5610
Epoch 6/100
7/7 [==============================] - 0s 13ms/step - loss: 1.0583 - accuracy: 0.7170 - val_loss: 1.1900 - val_accuracy: 0.5610
Epoch 7/100
7/7 [==============================] - 0s 17ms/step - loss: 1.0057 - accuracy: 0.7783 - val_loss: 1.1691 - val_accuracy: 0.5854
Epoch 8/100
7/7 [===